In [1]:
import numpy as np
import qutip as qt
import scipy.stats as sp
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import random
from ipynb.fs.full.myfunctions import *

In [155]:
def sample_ginibre_ensemble(n, p, dim_n, dim_k=None):
    # draw n states from the ginibre distribution (unbiased)
    if dim_k is None: dim_kk = dim_n
    else: dim_kk = dim_k
    x_0 = np.zeros((n, dim_n**2))
    w_0 = np.ones(n)/n
    for i in range(n):
        dm = rand_dm_ginibre(dim_n, dim_kk)
        x_0[i] = dm_to_bvector(dm, p, dim_n) # calculate pauli representation
    return x_0, w_0

def measurement(rho, c, dim):
    # create a measurement outcome of the state rho (in pauli representation) when measured in basis c
    prob = dim * np.sum(rho * c)
    if random.uniform(0, 1) < prob: res = 1
    else: res = 0
    return res

def experiment(rho, c, rep, dim):
    # measure rho pauli basises specified in c, repear ech measurement rep times, return # of measuered 1s
    c_rep = np.tile(c, [1, rep]).reshape((len(c) * rep, dim**2)) # repeated measurements
    d_rep = np.array([measurement(rho, c_i, dim) for c_i in c_rep]) # generate random measurements on rho     
    return np.sum(np.reshape(d_rep, (len(c), meas_rep)), axis= 1) # sum over measurement repetitions

def haarrand_measurement_basis(n, p, dim):
    # create n haar random bloch vectors of unit length
    c = np.zeros((n, dim**2))
    for i in range(n):
        dm = qt.ket2dm(qt.rand_ket_haar(dim))
        c[i] = dm_to_bvector(dm, p, dim)
    return c
    
def likelihood(x, d, c, rep, dim):
    # calculate likelihood of measurement outcome d given that the state is x
    p = [dim * np.sum(x_i * c) for x_i in x]
    return np.array([sp.binom.pmf(d, rep, p[i]) for i in range(len(x))])

def update_weights(x, w, d, c, rep, dim):
    # update weights according to likelihood and normalize, check whether resampling is required
    w_new = w * likelihood(x, d, c, rep, dim)
    w_new = np.divide(w_new, np.sum(w_new))
    return w_new

def pointestimate(x, w):
    # return point estimate of rho
    return np.average(x, axis=0, weights= w)

def fidelity(a, b, p):
    # compute fidelity from density matrices in Pauli representation
    return qt.metrics.fidelity(bvector_to_dm(a, p), bvector_to_dm(b, p))**2

In [58]:
n_q = 2 # number of Qubits - fixed in this implementation
dim = 2**n_q # Dimension of Hilbert space
p = create_pauli_basis(n_q) # create Pauli basis

# ideal state
rho = sample_ginibre_ensemble(1, p, dim, dim)[0][0]

# Experiment in Pauli basis
n_meas = 10 # number of measurements
meas_rep = 8 # number of repetitions of measurement
c_meas = haarrand_measurement_basis(n_meas, p, dim) # create bloch vectors of measurement basis

# sampling
n = 10001 # number of sampling points

In [59]:
# Experiments
d_meas = experiment(rho, c_meas, meas_rep, dim)

# Generate particle cloud according to unbiased ginibre prior
x, w = sample_ginibre_ensemble(n, p, dim, dim)

# Update the weights of the point cloud
for i in range(n_meas):
    w = update_weights(x, w, d_meas[i], c_meas[i], meas_rep, dim) 

In [60]:
# Estimates
rho_est = pointestimate(x, w)
print(fidelity(rho, rho_est, p))

0.8500228330726924
